## В этом домашнем задании вы сделаете первые шаги в мире линейной бинарной классификации!

In [1]:
import pandas as pd
import numpy as np
import warnings

#### Задание 1

Мы будем работать с данными **Microsoft Malware Detection**

Таргетом будет последний столбец `HasDetection`, который принимает значения $\{0,\, 1\}$ в случае отсутствия или наличия вируса на компьютере соответственно. Признаками будут выступать всевозможные характеристики компьютера.

In [103]:
data = pd.read_csv('train.csv')

C:\Users\egor2\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3397: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [55]:
data.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,...,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,...,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,...,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,...,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,0,7.0,0,NaN,53447.0,...,19844.0,0,0.0,0.0,0,0,0.0,0.0,1.0,1


Удалите константные признаки и признаки `ProductName` `MachineIdentifier`

In [86]:
### Your code is here


Посмотрите на соотношение классов в таргете. Все ли хорошо?

In [8]:
print(sum(data['HasDetections'] == 1), '- positive class,')
print(sum(data['HasDetections'] == 0), '- negative class')
print(sum(data['HasDetections'] == 1),',',sum(data['HasDetections'] == 0))

100060 - positive class,
99940 - negative class
100060 , 99940


Ответьте на вопрос: почему с вашей точки зрения важно иметь представление о балансе классов в ваших данных?

Избавьтесь от пропусков в данных! 

Новый для нас прием: признаки с более чем половиной пропусков следует удалить.

Согласитесь, если в вашей колонке среди 100 объектов всего лишь у 2 есть какое-то непропущенное значение, странно все остальные заполнять средним от этих двух чисел. Такие "редкие" признаки лучше вообще опустить!


В категориальных колонках заменим отсутствующую категорию просто некоторой новой и назовем ее `NaN`.

А в числовых, ради разнообразия, заполним пропуски медианным значением.

In [104]:
# удалить колонки с более чем 50% пропусков


In [105]:
data.shape

(200000, 73)

Создайте копию полученного датафрейма и положите ее в переменную data_2. Понадобится в следующих заданиях.

In [106]:
data_2 = data.copy()

Так же поработаем над всеми категориальными колонкам перед запуском непосредственно моделей.

Провернем самый базовый и наглый метод - несмотря на количество уникальных значений в каждой категории, просто применим ко всей категориальной части датасета `OneHotEncoding`

In [96]:
### Your code is here
from sklearn.preprocessing import OneHotEncoder

In [107]:
df_ohe = pd.get_dummies(data.select_dtypes(exclude=['float64', 'int64']), drop_first=True)
df_no_cat = data.select_dtypes(include=['float64', 'int64'])
data = pd.concat([df_ohe, df_no_cat], axis = 1)

In [108]:
data.shape

(200000, 6114)

###  Разделим выборку на тренировочную и тестовую

P.S. в задачах классификации, как и в задаче регрессии, можно использовать технологию Кросс-Валидации. Например, по одному из двух следующих сценариев:

1) Отделить валидацию и тест, произвести подбор лучшей модели с помощью `K-Fold` на валидации, финально обучить выбранную модель на всей валидации и замерить качество на заранее отложенном финальном тесте!

2) Всю выборку назвать валидационной и на ней применить `K-Fold` без финального замера.

В этой домашней работе попросим Вас быть еще проще! :)
Реализуем просто технологию отложенной выборки в пропорции 3:1

In [109]:
from sklearn.model_selection import train_test_split 

X = data.drop(columns=['HasDetections'])
y = data['HasDetections']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=1)

Соберите `Pipeline`, реализовав в нем 2 шага: стандартизация данных через `StandardScaler` и обучение логистической регрессии с помощью `LogisticRegression`, положите результаты в переменную `pipe`, а в классе модели `LogisticRegression` укажите параметр `penalty='none'`

In [110]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

### Your code is here
pipe = Pipeline([('Scaler', StandardScaler()), ('LogReg', LogisticRegression(penalty='none'))])


Чтобы замерить качество работы такой модели на трейне и на тесте воспользуемся функцией `cross_validate`

Вопрос: что передавать ей в параметр cv? Ведь мы уже разделили нашу выборку на трейн и тест (имеем всего 1 fold). Для этого можно просто передать список от кортежа, содержащего индексы тренировочных и тестовых объектов.

In [113]:
from sklearn.model_selection import cross_validate
import datetime

custom_cv = [(X_train.index.to_list(), X_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X, y, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)


print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")

Accuracy на трейне: nan
Accuracy на тесте: nan
Время работы алгоритма: 0:00:18.436575


C:\Users\egor2\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
1 fits failed out of a total of 1.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\egor2\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\egor2\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\egor2\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\pip

Что можете сказать про время работы алгоритма?

Очевидно, оно достаточно большие. Уж тем более для линейных моделей.

Такое как раз-таки происходит из-за того, что количество фичей, который мы передали нашей модели - гигантское! Классу требуется много времени и памяти, чтобы обработать датасет.

Поэтому те колонки, в которых количество уникальных категорий превышает какое-то адекватное число, следует кодировать иначе, нежели с помощью технологии `One-Hot-Encoding`.

Теперь вы верите, что более умные кодировки зачастую прям необходимы! Раньше мы этот факт не демонстрировали!

Дело еще вот в чем: в классе `LogisticRegression`, как и, например, `Lasso`, есть параметр, ограничивающий максимальное количество итераций во время обучения модели. Так, если данных много и итераций тоже ожидается большое число, найденная разделяющая гиперплоскость может оказаться не самой лучшей, так как наш алгоритм (будь то градиентный спуск или любой иной) просто 'не доползет'. 

#### Задание 2

Теперь попробуем другой метод кодирования категориальных колонок, а именно счётчики.
Построем ту же модель и на том же разделении, просто заново иначе переобработаем датасет. 

Для тех категориальных признаков, у которых количество уникальных значений в колоночках больше 5, применим `MeanTargetEncoding`.

Для остальных оставим любимый `OneHotEncoding` (как делали на практике и в предыдущем уроке).

In [127]:
### Your code is here


In [142]:
X_2 = data_2.drop(columns=['HasDetections'])
y_2 = data_2['HasDetections']
X_train, X_test, y_train, y_test = train_test_split(X_2, y_2,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=1)
pipe = Pipeline([('Scaler', StandardScaler()), ('LogReg', LogisticRegression(penalty='none'))])

Опять обучим модель, пока что без изменений! Скажите, стало ли быстрее? А что с качеством?

In [143]:
### Your code is here
custom_cv = [(X_train.index.to_list(), X_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X_2, y_2, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)


print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")

Accuracy на трейне: 0.638
Accuracy на тесте: 0.638
Время работы алгоритма: 0:00:07.053965


#### Задание 3: Регуляризация

Как и в моделях регрессии, решая задачу классификации, можем штрафовать минимизируемый функционал за большие веса, добавив к нему L1 или L2 норму весов (все как раньше!).

Для этого в изначальном классе `LogisticRegression` изменить параметр `penalty` на l1 или l2 соответственно. Выберите второй вариант! Можно воспользоваться методом `set_params` и применить его к `pipe`.

In [164]:
### Your code is here
pipe.get_params().keys()
# pipe.set_params(LogReg__penalty = 'l2')

dict_keys(['memory', 'steps', 'verbose', 'Scaler', 'LogReg', 'Scaler__copy', 'Scaler__with_mean', 'Scaler__with_std', 'LogReg__C', 'LogReg__class_weight', 'LogReg__dual', 'LogReg__fit_intercept', 'LogReg__intercept_scaling', 'LogReg__l1_ratio', 'LogReg__max_iter', 'LogReg__multi_class', 'LogReg__n_jobs', 'LogReg__penalty', 'LogReg__random_state', 'LogReg__solver', 'LogReg__tol', 'LogReg__verbose', 'LogReg__warm_start'])

Теперь наша модель будет строить логистическую регрессию с L2 регуляризатором! Помним, что у регуляризируемых моделей есть гиперпараметр, контролирующий силу регуляризации, который выбирается ДО запуска метода fit, то есть заранее, когда модель еще не обучена. 

Конечно же, выбор этого параметра влияет итоговые результаты. Хочется поперебирать различные параметры регуляризации и найти такой, при котором качество на тесте окажется лучшим! 

Сгенерируем массив гиперпараметра, которые планируем перебрать:

In [150]:
alphas = np.linspace(0.01, 100, 100)
alphas

array([1.000e-02, 1.020e+00, 2.030e+00, 3.040e+00, 4.050e+00, 5.060e+00,
       6.070e+00, 7.080e+00, 8.090e+00, 9.100e+00, 1.011e+01, 1.112e+01,
       1.213e+01, 1.314e+01, 1.415e+01, 1.516e+01, 1.617e+01, 1.718e+01,
       1.819e+01, 1.920e+01, 2.021e+01, 2.122e+01, 2.223e+01, 2.324e+01,
       2.425e+01, 2.526e+01, 2.627e+01, 2.728e+01, 2.829e+01, 2.930e+01,
       3.031e+01, 3.132e+01, 3.233e+01, 3.334e+01, 3.435e+01, 3.536e+01,
       3.637e+01, 3.738e+01, 3.839e+01, 3.940e+01, 4.041e+01, 4.142e+01,
       4.243e+01, 4.344e+01, 4.445e+01, 4.546e+01, 4.647e+01, 4.748e+01,
       4.849e+01, 4.950e+01, 5.051e+01, 5.152e+01, 5.253e+01, 5.354e+01,
       5.455e+01, 5.556e+01, 5.657e+01, 5.758e+01, 5.859e+01, 5.960e+01,
       6.061e+01, 6.162e+01, 6.263e+01, 6.364e+01, 6.465e+01, 6.566e+01,
       6.667e+01, 6.768e+01, 6.869e+01, 6.970e+01, 7.071e+01, 7.172e+01,
       7.273e+01, 7.374e+01, 7.475e+01, 7.576e+01, 7.677e+01, 7.778e+01,
       7.879e+01, 7.980e+01, 8.081e+01, 8.182e+01, 

Чтобы отобрать среди данного массива гиперпараметров лучший, воспользуйтесь конструкцией `GridSearchCV` из `sklearn`

In [165]:
from sklearn.model_selection import GridSearchCV

param_grid ={
    'LogReg__C': alphas
}

### Your code is here
g_s = GridSearchCV(estimator=pipe,
                   param_grid = param_grid,
                  verbose = 0)

In [167]:
g_s.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('Scaler', StandardScaler()),
                                       ('LogReg', LogisticRegression())]),
             param_grid={'LogReg__C': array([1.000e-02, 1.020e+00, 2.030e+00, 3.040e+00, 4.050e+00, 5.060e+00,
       6.070e+00, 7.080e+00, 8.090e+00, 9.100e+00, 1.011e+01, 1.112e+01,
       1.213e+01, 1.314e+01, 1.415e+01, 1.516e+01, 1.617e+01, 1.718e+01,
       1.819e+01, 1.920e+01, 2.021e+01, 2.122e+0...
       6.061e+01, 6.162e+01, 6.263e+01, 6.364e+01, 6.465e+01, 6.566e+01,
       6.667e+01, 6.768e+01, 6.869e+01, 6.970e+01, 7.071e+01, 7.172e+01,
       7.273e+01, 7.374e+01, 7.475e+01, 7.576e+01, 7.677e+01, 7.778e+01,
       7.879e+01, 7.980e+01, 8.081e+01, 8.182e+01, 8.283e+01, 8.384e+01,
       8.485e+01, 8.586e+01, 8.687e+01, 8.788e+01, 8.889e+01, 8.990e+01,
       9.091e+01, 9.192e+01, 9.293e+01, 9.394e+01, 9.495e+01, 9.596e+01,
       9.697e+01, 9.798e+01, 9.899e+01, 1.000e+02])})

In [174]:
pd.DataFrame(g_s.cv_results_).sort_values('mean_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogReg__C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
81,2.748057,0.261715,0.056253,0.007659,81.82,{'LogReg__C': 81.82000000000001},0.635533,0.640900,0.634033,0.637533,0.636567,0.636913,0.002306,100
78,2.864200,0.254347,0.053128,0.007654,78.79,{'LogReg__C': 78.79},0.635533,0.640900,0.634100,0.637533,0.636533,0.636920,0.002291,99
86,2.841081,0.234404,0.056254,0.007652,86.87,{'LogReg__C': 86.87},0.635500,0.640900,0.634167,0.637533,0.636533,0.636927,0.002279,89
89,2.835246,0.214535,0.050001,0.006256,89.9,{'LogReg__C': 89.9},0.635500,0.640900,0.634167,0.637533,0.636533,0.636927,0.002279,89
27,3.299658,0.243747,0.055136,0.007339,27.28,{'LogReg__C': 27.28},0.635533,0.640900,0.634100,0.637533,0.636567,0.636927,0.002290,89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,3.289244,0.290623,0.053148,0.004641,29.3,{'LogReg__C': 29.3},0.635600,0.640867,0.634200,0.637533,0.636667,0.636973,0.002242,4
46,3.279538,0.373550,0.057407,0.009160,46.47,{'LogReg__C': 46.47},0.635600,0.640900,0.634167,0.637533,0.636667,0.636973,0.002262,4
6,3.426757,0.065595,0.050228,0.000428,6.07,{'LogReg__C': 6.07},0.635533,0.640900,0.634167,0.637633,0.636667,0.636980,0.002276,3
12,3.364365,0.235808,0.053242,0.006114,12.13,{'LogReg__C': 12.13},0.635567,0.640900,0.634167,0.637667,0.636633,0.636987,0.002275,2


#### Задание 4: Бонус

Как вы знаете, подбор признаков является одной из самых важных частей машинного обучения. Сейчас вы попробуете на основе имеющихся признаков сгенерировать новые. В качестве новых признаков будем использовать мономы 2 степени. Можно использовать регуляризацию различного рода, выбор энкодера тоже за вами. Ваша задача - добиться качества `0.65` на тестовой выборке

In [2]:
import pandas as pd
data = pd.read_csv('train.csv')
data = data.drop(['ProductName', 'MachineIdentifier'], axis = 1)
data = data.dropna(axis = 1, thresh = len(data)/2)
cat_cols = data.select_dtypes(exclude=['float64', 'int64']).isnull().any()

for null_col in cat_cols[cat_cols].index:
    data[null_col] = data[null_col].fillna('NaN')
nans = np.sum(data.isna(), axis=0)
for col in list(nans[nans > 0].index):
    data[col].fillna(data[col].median(), inplace=True)
for i in data.columns:
    if (data[i] == data[i][0]).all():
        data.drop(columns=[i], inplace=True)

C:\Users\egor2\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3397: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
data_2 = data
catigorical = data_2.select_dtypes(exclude=['float64', 'int64'])
unic_cat = catigorical.describe().loc['unique']
col_for_ohe = unic_cat[unic_cat<=5].index.to_list() # столбцы с менее 5 уникальными признаками
df_ohe = pd.get_dummies(data_2[col_for_ohe], drop_first=True)
df_no_cat = data_2.drop(col_for_ohe, axis = 1)
data_2 = pd.concat([df_ohe, df_no_cat], axis = 1)
col_for_ohe = unic_cat[unic_cat>5].index.to_list() # столбцы с более 5 уникальными признаками
for column in col_for_ohe:
    mean_target = data_2.groupby(column)['HasDetections'].mean()
    data_2[column] = data_2[column].map(mean_target)

In [8]:
### Your code is here
from sklearn.preprocessing import PolynomialFeatures

In [20]:
X = data_2.drop('HasDetections', axis = 1)
y = data_2['HasDetections']
X.isna().sum().sum()

0

In [21]:
poly = PolynomialFeatures(degree=2)
poly.fit(X)
X = poly.transform(X)

MemoryError: Unable to allocate 5.07 GiB for an array with shape (200000, 3403) and data type float64

In [15]:
from sklearn.linear_model import LogisticRegressionCV

AttributeError: 'numpy.ndarray' object has no attribute 'isna'

In [18]:
model = LogisticRegressionCV(Cs=0.01, cv = 5, penalty='l2')
model.fit(X,y)

TypeError: object of type 'float' has no len()

In [ ]:
model.pre